In [ ]:
import numpy as np

In [ ]:
savez = np.load("/home/gnlenfn/data/remote/pytorch_emotion/Feature/data_save.npz", allow_pickle=True)

In [ ]:
savez['x'].shape
savez['y'].shape
savez['s'].shape
savez['t'].shape

In [ ]:
from collections import Counter
Counter(savez['y'])

In [ ]:
k = np.where(savez['y'] == 'exc', 'hap', savez['y'])
#savez['y'][exc] = 'hap'

In [ ]:
Counter(k)

In [ ]:
import pandas as pd
df = pd.read_csv("../Feature/emotion_classes.csv")
df.EMOTION.value_counts()                 

In [ ]:
k = []
for i in range(df.shape[0]):
    if df.TURN_NAME[i][7:12]=='impro':
        k.append(df.EMOTION[i])
        

In [ ]:
from collections import Counter
Counter(k)

In [ ]:
imp = np.where(savez['t'] == 'impro')[0]

In [ ]:
imp

In [ ]:
savez['t'][~imp]

In [ ]:
data = savez['x'][imp]

In [ ]:
k[imp].shape

In [ ]:
#exc = np.where(k[imp] == 'exc')[0]
hap = np.where(k[imp] == 'hap')[0]
ang = np.where(k[imp] == 'ang')[0]
neu = np.where(k[imp] == 'neu')[0]
sad = np.where(k[imp] == 'sad')[0]

In [ ]:
hap.shape[0] + ang.shape[0] + neu.shape[0] +sad.shape[0]

In [135]:
x_data = savez['x'][imp]
y_data = k[imp]

In [136]:
y_data.shape

(2942,)

In [ ]:
val = np.where((savez['s'] == "Ses05") & (savez['t'] == 'impro'))[0]

In [ ]:
val.shape

In [ ]:
val_idx = [x for x in range(x_data.shape[0])]

In [ ]:
ix = np.isin(val_idx, val)

In [137]:
x_data.shape

(2942,)

In [ ]:
ix.shape

In [106]:
speak = ['Ses01']#, 'Ses02', 'Ses03', 'Ses04', 'Ses05'] # cross validation
avg_acc = 0

is_train_set = True
for sp in speak:
    savez = np.load("/home/gnlenfn/data/remote/pytorch_emotion/Feature/data_save.npz", allow_pickle=True)
    total_idx = [x for x in range(savez['x'].shape[0])]
    if is_train_set == False:
        idx = np.where((savez['s'] == sp) & (savez['t'] == 'impro'))            
    else: idx = np.where((savez['s'] != sp) & (savez['t'] == 'impro'))

    x_data = savez['x'][idx]
    k = np.where(savez['y'] == 'exc', 'hap', savez['y'])
    y_data = k[idx]
    
    print(x_data.shape, y_data.shape)

(2421,) (2421,)


In [143]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_data = le.fit_transform(y_data)
y_data = to_categorical(y_data)

In [145]:
y_data

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [132]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical

In [ ]:
from keras import backend as K
K.expand_dims(x_data[0], axis=0).shape

In [152]:
c=0
for X, Y in zip(x_data, y_data):
    
    tmp = X.shape
    if tmp[0] == 0:
        print("x")
        print(c)
    c += 1
        
print(c)

x
532
2942


In [146]:
new_x = np.delete(x_data, 11)
new_y = np.delete(y_data, 11)

In [147]:
y_data

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
k = []
for i in range(len(new_x)):
    
    k.append(new_x[i].reshape(-1))
k = np.array(k)
X_train = k

In [149]:
model = Sequential()

model.add(LSTM(32, return_sequences=True, input_shape=(None, 32)))
#model.add(LSTM(8, return_sequences=True))
#model.add(TimeDistributed(Dense(1)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(Dense(4, activation='softmax'))

print(model.summary(90))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

for X, Y in zip(new_x, new_y):
    X = K.expand_dims(X, axis=0)
    Y = K.expand_dims(Y, axis=0)
    #print(Y.shape)
    model.fit(X, Y, verbose=1, steps_per_epoch=5)

Model: "sequential_50"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
lstm_66 (LSTM)                          (None, None, 32)                    8320          
__________________________________________________________________________________________
global_average_pooling1d_25 (GlobalAver (None, 32)                          0             
__________________________________________________________________________________________
dense_43 (Dense)                        (None, 4)                           132           
Total params: 8,452
Trainable params: 8,452
Non-trainable params: 0
__________________________________________________________________________________________
None
Epoch 1/1
5/5 [==============================] - 2s 301ms/step - loss: 0.9207
Epoch 1/1
5/5 [==============================] - 1s 137ms/step - loss: 0.6744
Epoch 1/1
5/5 [=====

InvalidArgumentError:  Tried to stack elements of an empty list with non-fully-defined element_shape: [?,32]
	 [[node lstm_66/TensorArrayV2Stack/TensorListStack (defined at /home/gnlenfn/anaconda3/envs/nn/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_378328]

Function call stack:
keras_scratch_graph


In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("best model: {}: {:4f}%".format(loaded_model.metrics_names[1], scores[1]*100))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn import preprocessing
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

In [ ]:
class FeatureData(Dataset):        
    def __init__(self, is_train_set=False, transforms=None):
        le = preprocessing.LabelEncoder()

        total_idx = [x for x in range(savez['x'].shape[0])]
        if is_train_set == False:
            idx = np.where((savez['s'] == "Ses01") & (savez['t'] == 'impro'))            
        else: idx = np.where((savez['s'] != "Ses01") & (savez['t'] == 'impro'))

        self.x_data = savez['x'][idx]
        k = np.where(savez['y'] == 'exc', 'hap', savez['y'])
        self.y_data = k[idx]
        self.y_data = le.fit_transform(self.y_data)
        self.len = self.x_data.shape[0]

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

    def __len__(self):
        return self.len

In [ ]:
batch_size = 1

In [ ]:
trans = transforms.Compose([transforms.ToTensor()])
train_dataset = FeatureData(is_train_set=False)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=False)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_dim, num_layers=1):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_dim, num_layers=self.num_layers)
        self.hidden = None
        
    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))
    
    def forward(self, inputs):
        _, self.hidden = self.lstm(inputs, self.hidden)
        return self.hidden.double()

In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_dim, num_layers=1):
        super(Decoder, self).__init__()
        # input size = 1
        self.lstm = nn.LSTM(4, hidden_dim, num_layers=num_layers)
        self.out = nn.Linear(hidden_dim, 4)
    def forward(self, outputs, hidden, criterion):
        batch_size, num_steps = outputs.shape
        input = torch.tensor([0.0] * batch_size, dtype=torch.double)
        input = input.unsqueeze(0)
        
        loss = 0
        for i in range(num_steps):
            # Push current input through LSTM: (seq_len=1, batch_size, input_size=1)
            output, hidden = self.lstm(input, hidden)
            # Push the output of last step through linear layer; returns (batch_size, 1)
            output = self.out(output[-1])
            # Generate input for next step by adding seq_len dimension (see above)
            input = output.unsqueeze(0)
            # Compute loss between predicted value and true value
            loss += criterion(output, outputs[:, i])
        return loss
    

In [ ]:
encoder = Encoder(32, 128)
decoder = Decoder(128)

encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
inputs = train_loader

In [ ]:
for X, Y in inputs:
    X = X.transpose(1,0)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    encoder.hidden = encoder.init_hidden(X.shape[1])
    
    hidden = encoder(X)
    
    loss = decoder(Y, hidden, criterion)
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    

In [166]:
speak = ['Ses01', 'Ses02', 'Ses03', 'Ses04', 'Ses05']
#for sp in speak:
    #print(sp)
    # DATA SET UP
le = LabelEncoder()
idx_test  = np.where((savez['s'] == 'Ses01') & (savez['t'] == 'impro'))
idx_train = np.where((savez['s'] != 'Ses01') & (savez['t'] == 'impro'))
    #train_set
x_train  = savez['x'][idx_train]    
y_train  = k[idx_train]
y_train  = le.fit_transform(y_train)    
    #test_set
x_test   = savez['x'][idx_test]    
y_test   = k[idx_test]
y_test   = le.fit_transform(y_test) 

c=0
for X, Y in zip(x_train, y_train):
    tmp = X.shape
    if tmp[0] == 0:
        print(sp)
        print("x")
        print(c)
        print(tmp)
    c += 1

print("test")
for X, Y in zip(x_test, y_test):
    tmp = X.shape
    if tmp[0] == 0:
        print(sp)
        print("x")
        print(c)
        print(tmp)
    c += 1

Ses05
x
11
(0, 32)
test


In [ ]:
new_x = np.delete(x_data, 11)
new_y = np.delete(y_data, 11)

In [161]:
x_train[11]

array([], shape=(0, 32), dtype=float64)

In [180]:
testt = idx_train[0]

In [190]:
k = []
for x in savez['x']:
    tmp = x.shape[0]
    k.append(tmp)
    
print(max(k))

3413


In [192]:
from keras.preprocessing import sequence

In [202]:
x_train = savez['x'][idx_train]
x_train = sequence.pad_sequences(x_train, maxlen=max(k), dtype='float')

In [203]:
x_train.shape

(2421, 3413, 32)

In [204]:
x_train[1]

array([[ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       ...,
       [ 2.933879e-04, -3.839608e+00, -3.601169e+00, ..., -1.583333e-03,
         1.606693e-02,  0.000000e+00],
       [ 3.002768e-04, -2.573599e+00, -2.166797e+00, ..., -3.916666e-03,
         1.507473e-02,  0.000000e+00],
       [ 3.190469e-04, -1.724054e+00, -1.454324e+00, ..., -2.833334e-03,
         1.049661e-02,  0.000000e+00]])

In [205]:
savez['x'][idx_train][1]

array([[ 6.891660e-04, -5.041446e+00, -4.809515e+00, ...,  2.499999e-03,
         7.128459e-03,  0.000000e+00],
       [ 6.640420e-04, -5.873871e+00, -5.339462e+00, ...,  4.833334e-03,
         7.748954e-03,  0.000000e+00],
       [ 6.499290e-04, -7.095003e+00, -5.808956e+00, ...,  8.166669e-03,
        -6.283715e-04,  0.000000e+00],
       ...,
       [ 2.933879e-04, -3.839608e+00, -3.601169e+00, ..., -1.583333e-03,
         1.606693e-02,  0.000000e+00],
       [ 3.002768e-04, -2.573599e+00, -2.166797e+00, ..., -3.916666e-03,
         1.507473e-02,  0.000000e+00],
       [ 3.190469e-04, -1.724054e+00, -1.454324e+00, ..., -2.833334e-03,
         1.049661e-02,  0.000000e+00]])

In [211]:
y_train  = to_categorical(le.fit_transform(y_train))

In [212]:
y_train.shape

(2421, 4)